In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 587, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 587 (delta 7), reused 0 (delta 0), pack-reused 570
Receiving objects: 100% (587/587), 13.21 MiB | 23.60 MiB/s, done.
Resolving deltas: 100% (347/347), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 27.9 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/hybrid-precision/tuning_hybrid_precision.db
/kaggle/input/hybrid-precision/__results__.html
/kaggle/input/hybrid-precision/__notebook__.ipynb
/kaggle/input/hybrid-precision/best_params_item_knn_precision.json
/kaggle/input/hybrid-precision/__output__.json
/kaggle/input/hybrid-precision/custom.css
/kaggle/input/hybrid-precision/Recsys_Challenge_2023/run_all_algorithms.py
/kaggle/input/hybrid-precision/Recsys_Challenge_2023/cython_example_FunkSVD.py
/kaggle/input/hybrid-precision/Recsys_Challenge_2023/LICENSE
/kaggle/input/hybrid-precision/Recsys_Challenge_2023/.gitignore
/kaggle/input/hybrid-precision/Recsys_Challenge_2023/rp3-tuning.ipynb
/kaggle/input/hybrid-precision/Recsys_Challenge_2023/README.md
/kaggle/input/hybrid-precision/Recsys_Challenge_2023/run_hyperparameter_search.py
/kaggle/input/hybrid-precision/Recsys_Challenge_2023/cython_example_SLIM_MSE.py
/kaggle/input/hybrid-precision/Recsys_Challenge_2023/requirements.txt
/kaggle/input/hybrid-precision/Recsys_Challe

In [5]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [6]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [7]:
config = {
    'tune_precision': False,
    'tune_hybrid': True,
    'database_path': '/kaggle/working/tuning_hybrid_precision.db',
    'copy_prev_best_params': True,
    'save_github': True
}

In [8]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_hybrid_precision.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

In [9]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [10]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [11]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [12]:
from Data_manager.IncrementalSparseMatrix import IncrementalSparseMatrix


class CrossValidationSplitter:
    def __init__(self, n_splits):
        self.n_splits = n_splits
        self.folds = []
        
    def split(self, URM):
        n_int_per_split = int(URM.nnz / self.n_splits)
        indices = np.arange(0, URM.nnz, 1)
        np.random.shuffle(indices)
        num_users, num_items = URM.shape
        
        URM_all = sps.coo_matrix(URM)
        
        for i in range(self.n_splits):
            if i == self.n_splits - 1:
                indices_valid = indices[n_int_per_split * i:]
                indices_train = indices[:n_int_per_split * i]
            else:
                indices_valid = indices[n_int_per_split * i: n_int_per_split * (i + 1)]
                indices_train_1 = indices[n_int_per_split * max(0, i - 1): n_int_per_split * i]
                indices_train_2 = indices[n_int_per_split * (i + 1):]
                indices_train = np.concatenate((indices_train_1, indices_train_2))
                
            URM_valid_builder = IncrementalSparseMatrix(
                n_rows=num_users, n_cols=num_items, auto_create_col_mapper=False, 
                auto_create_row_mapper=False
            )
            
            URM_train_builder = IncrementalSparseMatrix(
                n_rows=num_users, n_cols=num_items, auto_create_col_mapper=False, 
                auto_create_row_mapper=False
            )
            
            URM_valid_builder.add_data_lists(
                URM_all.row[indices_valid],
                URM_all.col[indices_valid],
                URM_all.data[indices_valid]
            )
            
            URM_train_builder.add_data_lists(
                URM_all.row[indices_train],
                URM_all.col[indices_train],
                URM_all.data[indices_train]
            )
            
            yield URM_train_builder.get_SparseMatrix(), URM_valid_builder.get_SparseMatrix()

In [13]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [14]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

In [15]:
from concurrent.futures import ThreadPoolExecutor

def evaluate_item_knn(URM_train, URM_valid, params):
    recommender = ItemKNNCFRecommender(URM_train)
    recommender.fit(**params)

    evaluator = EvaluatorHoldout(URM_valid, cutoff_list=[100])
    result_df, _ = evaluator.evaluateRecommender(recommender)
    print(f'Precision is {result_df.loc[100, "PRECISION"]}')
    return result_df.loc[100, 'PRECISION']


def objective_item_knn(trial):
    params = {
        'topK': trial.suggest_int('topK', 10, 1000),
        'shrink': trial.suggest_int('shrink', 0, 500),
        'similarity': trial.suggest_categorical('similarity', ["cosine", "asymmetric", "dice", "jaccard", "tanimoto", "tversky", "euclidean"]),        
    }
    
    if params['similarity'] == "asymmetric":
        params["asymmetric_alpha"] = trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        params["normalize"] = True     

    elif params['similarity'] == "tversky":
        params["tversky_alpha"] = trial.suggest_float("tversky_alpha", 0, 2, log=False)
        params["tversky_beta"] = trial.suggest_float("tversky_beta", 0, 2, log=False)
        params["normalize"] = True 

    elif params['similarity'] == "euclidean":
        params["normalize_avg_row"] = trial.suggest_categorical("normalize_avg_row", [True, False])
        params["similarity_from_distance_mode"] = trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        params["normalize"] = trial.suggest_categorical("normalize", [True, False])
    
    results = [None] * 10
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        for URM_train, URM_valid in CrossValidationSplitter(10).split(URM_all):
            futures.append(executor.submit(evaluate_item_knn, URM_train, URM_valid, params))
        print(len(futures))
        for j, future in enumerate(futures):
            results[j] = future.result()
        
    return np.mean(results)

In [16]:
if config['tune_precision']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_knn_item_precision@100', 
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective_item_knn, n_trials=50)

In [17]:
if config['tune_precision']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [18]:
if config['tune_precision']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [19]:
if config['tune_precision']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [20]:
if config['tune_precision']:
    with open('/kaggle/working/best_params_item_knn_precision.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_item_knn_precision.json', 
            'tuning_results/best_params_item_knn_precision.json', 
            'Item KNN precision results (from kaggle notebook)'
        )
    
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/input/hybrid-precision/best_params_item_knn_precision.json', 
        '/kaggle/working/best_params_item_knn_precision.json'
    )

In [21]:
with open('/kaggle/working/best_params_item_knn_precision.json', 'r') as params_file:
    params_knn = json.load(params_file)
params_knn

{'topK': 32, 'shrink': 33, 'similarity': 'tanimoto'}

In [22]:
recommender = ItemKNNCFRecommender(URM_train)
recommender.fit(**params_knn)

evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])
result_df, _ = evaluator.evaluateRecommender(recommender)
result_df

ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 4915.70 column/sec. Elapsed time 4.55 sec
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 8.20 sec. Users per second: 1274


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.085033,0.147081,0.123712,0.043282,0.0743,0.248788,0.135035,0.100789,0.482881,0.324451,...,0.802826,0.387669,0.802826,0.054251,10.03004,0.997326,0.156536,0.772675,1.925012,0.295821


In [23]:
URM_train_item_knn = URM_train.dot(recommender.W_sparse)
URM_train_item_knn

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 9768701 stored elements in Compressed Sparse Row format>

In [24]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_RP3.json', 'r') as params_file:
    params_rp3 = json.load(params_file)
params_rp3

{'alpha': 0.513297287731911, 'beta': 0.14806602610935488, 'topK': 54}

In [25]:
recommender_rp3 = RP3betaRecommender(URM_train_item_knn)
recommender_rp3.fit(**params_rp3)

RP3betaRecommender: URM Detected 629 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 248 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 192.53 column/sec. Elapsed time 1.93 min


In [26]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10], exclude_seen=False)
result_df, _ = evaluator.evaluateRecommender(recommender_rp3)
result_df

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 9.84 sec. Users per second: 1062


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.040149,0.080732,0.071836,0.014701,0.029663,0.112219,0.062321,0.05151,0.309296,0.128398,...,0.802826,0.248311,0.802826,0.015741,8.70746,0.994924,0.035004,0.642622,2.024639,0.011825


In [27]:
class PipelineRecommender:
    def __init__(self, coarse_recommender_class, fine_recommender_class, coarse_fit_params, fine_fit_params, URM_train):
        self.URM_train = URM_train
        self.coarse_recommender_class = coarse_recommender_class
        self.fine_recommender_class = fine_recommender_class
        self.coarse_fit_params = coarse_fit_params
        self.fine_fit_params = fine_fit_params
    
    def fit(self):
        self.coarse_recommender = self.coarse_recommender_class(self.URM_train)
        self.coarse_recommender.fit(**self.coarse_fit_params)
        URM_coarse = self.URM_train.dot(self.coarse_recommender.W_sparse)
        self.fine_recommender = self.fine_recommender_class(URM_coarse)
        self.fine_recommender.fit(**self.fine_fit_params)
        
    def recommend(self, user_ids, cutoff=10, remove_seen_flag=True, remove_top_pop_flag=False, 
                  return_scores=True, remove_custom_items_flag=True):
        items = set(np.arange(self.URM_train.shape[1]).tolist())
        final_recommendations = []
        final_scores = []
        for user_id in user_ids:
            # custom remove seen flag, since the fine recommender is trained with much more non zero entries
            # but the true seen items are in the original urm
            items_to_compute = list(items - set(self.URM_train[user_id].nonzero()[1].tolist()))
            recommendations, scores = self.fine_recommender.recommend(
                [user_id],
                cutoff=cutoff,
                items_to_compute=items_to_compute, 
                remove_seen_flag=False, # if true it will remove most of the items since fine train matrix is less sparse
                remove_top_pop_flag=remove_top_pop_flag, 
                return_scores=True, 
                remove_custom_items_flag=remove_custom_items_flag
            )
            final_recommendations.append(recommendations[0])
            final_scores.append(scores[0])
        if return_scores:
            return np.array(final_recommendations), np.array(final_scores)
        return np.array(final_recommendations)
        
    def get_URM_train(self):
        return self.URM_train

In [28]:
recommender = PipelineRecommender(ItemKNNCFRecommender, RP3betaRecommender, params_knn, params_rp3, URM_train)
recommender.fit()

ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 5027.78 column/sec. Elapsed time 4.44 sec
RP3betaRecommender: URM Detected 629 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 248 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 191.69 column/sec. Elapsed time 1.94 min


In [29]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])
result_df, _ = evaluator.evaluateRecommender(recommender)
result_df

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 54.33 sec. Users per second: 192


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.057785,0.105179,0.08944,0.025882,0.045513,0.164249,0.089164,0.070209,0.374904,0.204495,...,0.802826,0.300983,0.802826,0.018422,8.910796,0.995616,0.053154,0.686453,2.002961,0.289269


In [30]:
folds = []
for URM_train, URM_valid in CrossValidationSplitter(5).split(URM_all):
    recommender = ItemKNNCFRecommender(URM_train)
    recommender.fit(**params_knn)
    
    folds.append((URM_train, URM_valid, recommender))

ItemKNNCFRecommender: URM Detected 631 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 248 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 5027.23 column/sec. Elapsed time 4.45 sec
ItemKNNCFRecommender: URM Detected 599 ( 4.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 228 ( 1.0%) items with no interactions.
Similarity column 22347 (100.0%), 5084.25 column/sec. Elapsed time 4.40 sec
ItemKNNCFRecommender: URM Detected 944 ( 7.2%) users with no interactions.
ItemKNNCFRecommender: URM Detected 545 ( 2.4%) items with no interactions.
Similarity column 22347 (100.0%), 5501.97 column/sec. Elapsed time 4.06 sec
ItemKNNCFRecommender: URM Detected 1462 (11.2%) users with no interactions.
ItemKNNCFRecommender: URM Detected 1494 ( 6.7%) items with no interactions.
Similarity column 22347 (100.0%), 5725.55 column/sec. Elapsed time 3.90 sec
ItemKNNCFRecommender: URM Detected 599 ( 4.6%) users with no interactions.
ItemKNNCFRecommende

In [31]:
class PrefittedPipelineRecommender:
    def __init__(self, fine_recommender, coarse_recommender, URM_train):
        self.URM_train = URM_train
        self.coarse_recommender = coarse_recommender
        self.fine_recommender = fine_recommender
    
    def fit(self):
        pass
        
    def recommend(self, user_ids, cutoff=10, remove_seen_flag=True, remove_top_pop_flag=False, 
                  return_scores=True, remove_custom_items_flag=True):
        items = set(np.arange(self.URM_train.shape[1]).tolist())
        final_recommendations = []
        final_scores = []
        for user_id in user_ids:
            # custom remove seen flag, since the fine recommender is trained with much more non zero entries
            # but the true seen items are in the original urm
            items_to_compute = list(items - set(self.URM_train[user_id].nonzero()[1].tolist()))
            recommendations, scores = self.fine_recommender.recommend(
                [user_id],
                cutoff=cutoff,
                items_to_compute=items_to_compute, 
                remove_seen_flag=False, # if true it will remove most of the items since fine train matrix is less sparse
                remove_top_pop_flag=remove_top_pop_flag, 
                return_scores=True, 
                remove_custom_items_flag=remove_custom_items_flag
            )
            final_recommendations.append(recommendations[0])
            final_scores.append(scores[0])
        if return_scores:
            return np.array(final_recommendations), np.array(final_scores)
        return np.array(final_recommendations)
        
    def get_URM_train(self):
        return self.URM_train

In [32]:
from concurrent.futures import ThreadPoolExecutor

def evaluate_pipeline_recommender(URM_train, URM_valid, item_knn_recommender, params):
    recommender = RP3betaRecommender(URM_train.dot(item_knn_recommender.W_sparse))
    recommender.fit(**params)
    hybrid_recommender = PrefittedPipelineRecommender(recommender, item_knn_recommender, URM_train)

    evaluator = EvaluatorHoldout(URM_valid, cutoff_list=[10])
    result_df, _ = evaluator.evaluateRecommender(hybrid_recommender)
    print(f'MAP is {result_df.loc[10, "MAP"]}')
    
    return result_df.loc[10, 'MAP']


def objective_item_knn(trial):
    params = {
        'alpha': trial.suggest_float('alpha', 0.01, 10, log=True),
        'beta': trial.suggest_float('beta', 0.01, 10, log=True),
        'topK': trial.suggest_int('topK', 10, 5000, log=True)
    }
    
    results = [None] * 5
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = []
        for URM_train, URM_valid, item_knn_recommender in folds:
            futures.append(executor.submit(evaluate_pipeline_recommender, URM_train, URM_valid, item_knn_recommender, params))
        for j, future in enumerate(futures):
            results[j] = future.result()
            
    gc.collect()
        
    return np.mean(results)

In [33]:
if config['tune_hybrid']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_stag2_rp3', 
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective_item_knn, n_trials=20)

[I 2023-11-24 14:28:25,235] A new study created in RDB with name: hyperparameters_tuning_stag2_rp3


RP3betaRecommender: URM Detected 1482 (11.4%) users with no interactions.
RP3betaRecommender: URM Detected 951 ( 7.3%) users with no interactions.
RP3betaRecommender: URM Detected 1512 ( 6.8%) items with no interactions.
RP3betaRecommender: URM Detected 551 ( 2.5%) items with no interactions.
RP3betaRecommender: URM Detected 634 ( 4.9%) users with no interactions.
RP3betaRecommender: URM Detected 602 ( 4.6%) users with no interactions.
RP3betaRecommender: URM Detected 250 ( 1.1%) items with no interactions.
RP3betaRecommender: URM Detected 601 ( 4.6%) users with no interactions.
RP3betaRecommender: URM Detected 231 ( 1.0%) items with no interactions.
RP3betaRecommender: URM Detected 235 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 95.64 column/sec. Elapsed time 3.89 min
RP3betaRecommender: Similarity column 22347 (100.0%), 83.23 column/sec. Elapsed time 4.47 min
EvaluatorHoldout: Ignoring 2539 (19.5%) Users that have less than 1 test interac

[I 2023-11-24 14:40:36,282] Trial 0 finished with value: 0.023229795498854868 and parameters: {'alpha': 2.534006571935572, 'beta': 0.2850493891688343, 'topK': 919}. Best is trial 0 with value: 0.023229795498854868.


MAP is 0.026046204245918286
RP3betaRecommender: URM Detected 1482 (11.4%) users with no interactions.
RP3betaRecommender: URM Detected 951 ( 7.3%) users with no interactions.
RP3betaRecommender: URM Detected 1512 ( 6.8%) items with no interactions.
RP3betaRecommender: URM Detected 551 ( 2.5%) items with no interactions.
RP3betaRecommender: URM Detected 634 ( 4.9%) users with no interactions.
RP3betaRecommender: URM Detected 601 ( 4.6%) users with no interactions.
RP3betaRecommender: URM Detected 602 ( 4.6%) users with no interactions.
RP3betaRecommender: URM Detected 250 ( 1.1%) items with no interactions.
RP3betaRecommender: URM Detected 235 ( 1.1%) items with no interactions.
RP3betaRecommender: URM Detected 231 ( 1.0%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 192.39 column/sec. Elapsed time 1.94 min
EvaluatorHoldout: Ignoring 2539 (19.5%) Users that have less than 1 test interactions
RP3betaRecommender: Similarity column 22347 (100.0%), 140.6

[I 2023-11-24 14:49:04,037] Trial 1 finished with value: 0.0015699140421317006 and parameters: {'alpha': 1.60726651686522, 'beta': 2.550402096047494, 'topK': 71}. Best is trial 0 with value: 0.023229795498854868.


...

[I 2023-11-24 18:13:57,673] Trial 17 finished with value: 0.02002190186845353 and parameters: {'alpha': 3.1091350742757458, 'beta': 0.531270825669978, 'topK': 912}. Best is trial 16 with value: 0.02390904223420983.


RP3betaRecommender: URM Detected 1482 (11.4%) users with no interactions.
RP3betaRecommender: URM Detected 951 ( 7.3%) users with no interactions.
RP3betaRecommender: URM Detected 1512 ( 6.8%) items with no interactions.
RP3betaRecommender: URM Detected 602 ( 4.6%) users with no interactions.
RP3betaRecommender: URM Detected 634 ( 4.9%) users with no interactions.
RP3betaRecommender: URM Detected 601 ( 4.6%) users with no interactions.
RP3betaRecommender: URM Detected 551 ( 2.5%) items with no interactions.
RP3betaRecommender: URM Detected 231 ( 1.0%) items with no interactions.
RP3betaRecommender: URM Detected 235 ( 1.1%) items with no interactions.
RP3betaRecommender: URM Detected 250 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 74.51 column/sec. Elapsed time 5.00 min
RP3betaRecommender: Similarity column 17400 (77.9%), 58.00 column/sec. Elapsed time 5.00 min
RP3betaRecommender: Similarity column 17800 (79.7%), 59.21 column/sec. Elapsed ti

[I 2023-11-24 18:29:12,066] Trial 18 finished with value: 0.01098668584414791 and parameters: {'alpha': 4.352649530119464, 'beta': 1.105741323369299, 'topK': 1425}. Best is trial 16 with value: 0.02390904223420983.


RP3betaRecommender: URM Detected 951 ( 7.3%) users with no interactions.
RP3betaRecommender: URM Detected 1482 (11.4%) users with no interactions.
RP3betaRecommender: URM Detected 551 ( 2.5%) items with no interactions.
RP3betaRecommender: URM Detected 1512 ( 6.8%) items with no interactions.
RP3betaRecommender: URM Detected 602 ( 4.6%) users with no interactions.
RP3betaRecommender: URM Detected 601 ( 4.6%) users with no interactions.
RP3betaRecommender: URM Detected 634 ( 4.9%) users with no interactions.
RP3betaRecommender: URM Detected 250 ( 1.1%) items with no interactions.
RP3betaRecommender: URM Detected 231 ( 1.0%) items with no interactions.
RP3betaRecommender: URM Detected 235 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 108.10 column/sec. Elapsed time 3.45 min
EvaluatorHoldout: Ignoring 2539 (19.5%) Users that have less than 1 test interactions
RP3betaRecommender: Similarity column 22347 (100.0%), 91.66 column/sec. Elapsed time 4.

[I 2023-11-24 18:40:15,469] Trial 19 finished with value: 0.008965879292099748 and parameters: {'alpha': 9.566801029047626, 'beta': 0.3231454532611822, 'topK': 516}. Best is trial 16 with value: 0.02390904223420983.


In [34]:
if config['tune_hybrid']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [35]:
if config['tune_hybrid']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [36]:
if config['tune_hybrid']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [37]:
if config['save_github'] and (config['tune_precision'] or config['tune_hybrid']):
    upload_file(
        config['database_path'],
        'tuning_results/tuning_hybrid_precision.db', 
        'Hybrid pipeline results(from kaggle notebook)'
    )